# Perform forward kinematics with a trajectory

In this notebook, we are going to generate a short video for a random robot movement you can watch in your browser.

First of all, we need some modules. You can create some simple modules built from geometric primitives yourself - there is a distinct tutorial to that purpose. If you don't want to, we can use the set of simple modules provided with the timor source code:

In [13]:
from pathlib import Path

# provide your own filepaths if you already did the module generation tutorial:
your_modules_json = Path('')
your_assets_directory = Path('')

if your_modules_json == Path(''):
    from timor.utilities.file_locations import get_module_db_files
    modules_file = get_module_db_files('geometric_primitive_modules')

In [14]:
import numpy as np
import timor
from timor.Module import *
from timor.utilities.visualization import animation, MeshcatVisualizerWithAnimation

#create the DB object
db = ModulesDB.from_json_file(modules_file)

#print out available modules in DB
print(db.by_name)
print(db.by_id)
# print(db.all_joints)
# print(db.all_connectors)


{'Revolute Joint': <timor.Module.AtomicModule object at 0x7f97c8f82020>, 'I shaped link 0.08-0.08-15': <timor.Module.AtomicModule object at 0x7f97c8e114b0>, 'I shaped link 0.08-0.08-45': <timor.Module.AtomicModule object at 0x7f97c90192d0>, 'L shaped link 0.1-0.08-15': <timor.Module.AtomicModule object at 0x7f97c901a0b0>, 'L shaped link 0.1-0.08-30': <timor.Module.AtomicModule object at 0x7f97c901a470>, 'Base': <timor.Module.AtomicModule object at 0x7f97c8fe2620>, 'Prismatic Joint': <timor.Module.AtomicModule object at 0x7f97c8e838e0>, 'L shaped link 0.1-0.08-45': <timor.Module.AtomicModule object at 0x7f97c901de70>, 'I shaped link 0.08-0.08-30': <timor.Module.AtomicModule object at 0x7f97caa9a410>, 'Demo EEF': <timor.Module.AtomicModule object at 0x7f97c8f59bd0>}
{'J2': <timor.Module.AtomicModule object at 0x7f97c8f82020>, 'i_15': <timor.Module.AtomicModule object at 0x7f97c8e114b0>, 'i_45': <timor.Module.AtomicModule object at 0x7f97c90192d0>, 'l_15': <timor.Module.AtomicModule objec

In [15]:
#contains IDs we care about. The order is arbitrary, but if we want to make a robot out of this (i.e. kinematic tree) we should put in order.
modules = ('base', 'J2', 'i_45', 'J2', 'J2', 'eef')
A = ModuleAssembly.from_serial_modules(db, modules)
robot = A.to_pin_robot() #convert to pinocchio robot

viz = robot.visualize()
viz.viewer.jupyter_cell(height=400) # generate a jupyter cell to visualize the robot

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [16]:

#initial, final states
q0 = np.zeros((robot.dof,))
print(f"Robot has {robot.dof} joints.")
print("Robot joint state min limits: ", robot.joint_limits[0])
print("Robot joint state max limits: ", robot.joint_limits[1])

q1 = [float(x) for x in input("Put in a new joint state: ").split(' ')]
print(f"New joint state set to {q1}")
# q1 = robot.random_configuration()

#make a constant-time trajectory
trajectory = np.linspace(q0, q1)

#create animation - https://timor-python.readthedocs.io/en/stable/autoapi/timor/utilities/visualization/index.html#timor.utilities.visualization.animation
viz_A = animation(robot, trajectory, dt=.1)
viz_A.viewer.jupyter_cell(height=400) # generate a jupyter cell to visualize the robot

Robot has 3 joints.
Robot joint state min limits:  [-3.14159265 -3.14159265 -3.14159265]
Robot joint state max limits:  [3.14159265 3.14159265 3.14159265]
New joint state set to [5.0]
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


In [17]:
#arbitrary new joint state
q2 = np.array([np.pi, -np.pi/2, np.pi/2])


"""Forwards Kinematics (FK) and Forwards Dynamics (FD) methods belong to PinRobot object"""
  # FK https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fk
  # FD https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fd

# FK calculates the relative rigid body transform from the reference config at a given joint angle position. 
# You can tell it to update the collision hitbox and visual mesh if wanted. I don't think this robot can collide with itself.
print(f"Robot end effector coordinate transform at joint state {q2}", robot.fk(configuration = q2, collision = True, visual = True)) #default kind is tcp - tool center point
print(f"Robot joint coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'joints', collision = True, visual = True))
print(f"Robot full frames coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'full', collision = True, visual = True))


#visual just updates the state of the visual geometries, we have to actually visualize to see it
robot.visualize()

#Current robot velocities
print(f"Current robot joint positions (configuration): {robot.configuration}")
print(f"Current robot joint velocities: {robot.velocities}")

# FD calculates joint accelerations (ddq) given joint config (q) and joint velocity (dq) 
print(robot.fd(tau = [1, 0, 0], motor_inertia = False, friction = True))
  

Robot end effector coordinate transform at joint state [ 3.14159265 -1.57079633  1.57079633] [[-0.    1.   -0.    0.5 ]
 [-0.   -0.   -1.   -0.19]
 [-1.   -0.    0.    0.8 ]
 [ 0.    0.    0.    1.  ]]
Robot joint coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[ 1.    0.    0.    0.2 ]
 [-0.    1.    0.   -0.  ]
 [ 0.   -0.    1.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.   0.   1.   0.2]
 [-1.  -0.  -0.   0. ]
 [ 0.  -1.   0.   0.8]
 [ 0.   0.   0.   1. ]], [[-0.   1.  -0.   0.5]
 [-0.  -0.  -1.   0. ]
 [-1.  -0.   0.   0.8]
 [ 0.   0.   0.   1. ]])
Robot full frames coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[ 1.  0.  0.  0.]
 [ 0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]], [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[-0.   -0.    1.    0.05]
 [ 0.   -1.    0.    0.  ]
 [ 1.    0.    0.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.    0

In [18]:
"""Example of manually defined trajectory with Forwards Kinematics (pt 1)"""

modules = ('base', 'i_30', 'J2', 'J2', 'J2', 'i_30', 'eef')
B = ModuleAssembly.from_serial_modules(db, modules)
long_robot = B.to_pin_robot() #convert to pinocchio robot

q_0 = [np.pi, np.pi, np.pi] #start in this config with no collisions
long_robot.fk(q_0, collision = True, visual = True)

#the collisions() method exists, but requires the definition of a task. we look at collision pairs as a quick shortcut.
print("Self collision in this state: ", long_robot.has_self_collision())
long_robot.visualize_self_collisions() #nothing is highlighted b/c no self collisions

Self collision in this state:  False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [19]:
"""Example of manually defined trajectory with Forwards Kinematics (pt 2)"""

q_1 = [0, 0, 0] #this ending config has collisions
collision_trajectory = np.linspace(q_0, q_1)

for x in collision_trajectory:
    #update the forward kinematics to the next joint state. 
    # We could try manually setting joint state and updating the collision and visual meshes, but this is cleaner. 
    fk_transform = long_robot.fk(x, collision=True, visual=True)

    self_collision = long_robot.has_self_collision()
    #print out collisions at this state
    if self_collision:
        print(f"Self-collision detected at joint state {x}.")

    long_robot.update_configuration(x)
    viz.updatePlacements(timor.visualization.VISUAL)  # We need to tell the visualizer to update
    # viz.viewer.jupyter_cell(height = 800)

# Visualize self collisions at 0 config (end of trajectory)
long_robot.visualize_self_collisions()

Self-collision detected at joint state [0.12822827 0.12822827 0.12822827].
Self-collision detected at joint state [0.06411414 0.06411414 0.06411414].
Self-collision detected at joint state [0. 0. 0.].
collision pair detected: 1 , 11 - collision: Yes
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/


In [22]:
# animate the whole trajectory and visualize it
# tip: open the control setting on the upper right corner of the jupyter cell. Under animation, you can play and pause the animation.
viz_B = animation(long_robot, collision_trajectory, dt=.1)
viz_B.viewer.jupyter_cell(height=400)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/


In [10]:
#Inverse Kinematics and Dynamics - TODO